### Question1_a
- Visiualize the data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
amp_data = np.load('amp_data.npz')['amp_data']
plt.figure(figsize=(18, 10))
plt.subplot(2, 1, 1)
plt.plot(amp_data)
plt.title('Line graph')
plt.ylabel('Value of amp_data')
plt.xlabel('Index')
plt.subplot(2, 1, 2)
plt.hist(amp_data,bins=100)
plt.title('Histogram')
plt.xlabel('amp_data')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

### Question1_b
- Dataset split

In [ ]:
row=int(np.floor(amp_data.shape[0]/21)) # 33713280=21*1605394+6
amp_data_new=amp_data[:row*21].reshape(row,21)
np.random.seed(20211011)
amp_data_perm=np.random.permutation(amp_data_new)


[train,val,test,]=np.vsplit(amp_data_perm,[1123776,1364585,1605394])[:3]
X_shuf_train=train[:,:20]
y_shuf_train=train[:,20]
X_shuf_val=val[:,:20]
y_shuf_val=val[:,20]
X_shuf_test=test[:,:20]
y_shuf_test=test[:,20]

### Question2_a
- Line fit and quartic fit

In [ ]:
t_x=np.linspace(0,1,21,endpoint=True)[:-1][:,None]
t_y=np.linspace(0,1,21,endpoint=True)[-1]

def q2_phi (t):
    return np.hstack([np.ones(t.shape[0])[:,None],t,t**2,t**3,t**4])

def q2_line (t):
    return np.hstack([np.ones(t.shape[0])[:,None],t])

def q2_plot(i):
    # i is the row index of training data matrices
    xx=X_shuf_train[i,:][:,None]
    yy=y_shuf_train[i]
    
    # Compute the fitted parameter of line fit and quartic fit
    w_q2a_line=np.linalg.lstsq(q2_line(t_x),xx,rcond=None)[0]
    w_q2a_quartic=np.linalg.lstsq(q2_phi(t_x),xx,rcond=None)[0]
    # Plot the training points and the test point
    plt.scatter(t_x,xx,label='Training point')
    plt.plot(t_y,yy,'ko',label='Test point')

    t_grid=np.linspace(0,1,1000,endpoint=True)[:,None]

    plt.plot(t_grid,np.dot(q2_line(t_grid),w_q2a_line),linewidth=2,color='g',label='Line fit')
    plt.plot(t_y,np.dot(q2_line(t_grid),w_q2a_line)[-1],marker='o',color='g',label='Predicted point of line fit')
    plt.text(t_y,np.dot(q2_line(t_grid),w_q2a_line)[-1],str(np.dot(q2_line(t_grid),w_q2a_line)[-1]),color='g')
    
    plt.plot(t_grid,np.dot(q2_phi(t_grid),w_q2a_quartic),linewidth=2,color='r',label='Quartic fit')
    plt.plot(t_y,np.dot(q2_phi(t_grid),w_q2a_quartic)[-1],marker='o',color='r',label='Predicted point of quartic fit')
    plt.text(t_y,np.dot(q2_phi(t_grid),w_q2a_quartic)[-1],str(np.dot(q2_phi(t_grid),w_q2a_quartic)[-1]),color='r')
    plt.title('Q2 (a)')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    
    plt.legend(bbox_to_anchor=(1.15, 1.0), loc='upper left')
    plt.show()
    
q2_plot(0)    

### Question2_c
- Line fit and quartic fit

In [ ]:
q2_plot(1)
q2_plot(-1)

### Question3_b
- Prediction

In [ ]:
def Phi(C,K):
    t_q3_train=np.linspace(0,1,21)[:-1] 
    t=t_q3_train[(20-C):][:,None]
    phi=np.ones(C)[:,None]
    for i in range(1,K):
        phi=np.hstack([phi,t**i])
    return phi

def make_vv(C, K):
    phi_phi=np.linalg.inv(np.dot(Phi(C,K).T,Phi(C,K)))
    phi_t1=np.ones(1)[:,None]
    t=np.array([1])[:,None] #t=1
    for i in range(1,K):
        phi_t1=np.vstack([phi_t1,t**i])
    return np.dot(Phi(C,K),phi_phi).dot(phi_t1) # return a Cx1 vector

print('Predictions at t=1 in Q3:\n Linear:{0:f} \n Quartic:{1:f}'
      .format(float(np.dot(make_vv(20,2).T,X_shuf_train[0,:][:,None]))
              ,float(np.dot(make_vv(20,5).T,X_shuf_train[0,:][:,None]))))

### Question3_c
- Cross Validation

In [ ]:
cmax=20;kmax=6;
se_train=np.zeros((cmax,kmax))
for c in range(cmax):
    for k in range(kmax):
        pred=make_vv(c+1,k+1).T@X_shuf_train[:,(19-c):].T
        se_train[c,k]=np.sum((y_shuf_train-pred)**2)
        
min_index=np.asarray(np.where(se_train==np.min(se_train))) 
print('C=%i,K=%i'%(int(min_index[0])+1,int(min_index[1])+1))

# prediction of validation set
pred_val=np.dot(make_vv(2,2).T,X_shuf_val[:,18:].T)
# mse on the validation set
mse_val=((y_shuf_val-pred_val)**2).sum()/X_shuf_val.shape[0]

# prediction of test set
pred_test=np.dot(make_vv(2,2).T,X_shuf_test[:,18:].T)
# mse on the test set
mse_test=((y_shuf_test-pred_test)**2).sum()/X_shuf_test.shape[0]
print('MSE:\nTraining set:%.4e \nValidation set:%.4e \nTest set:%.4e'
      %(np.min(se_train)/X_shuf_train.shape[0],mse_val,mse_test))

### Question4_a
- Cross Validation

In [ ]:
cmax=20;
mse_train_q4=np.zeros(cmax)
mse_val_q4=np.zeros(cmax)
for c in range(cmax):
    v_train_fit=np.linalg.lstsq(X_shuf_train[:,(19-c):],y_shuf_train,rcond=None)[0]
    
    pred_train_q4=np.dot(X_shuf_train[:,(19-c):],v_train_fit)
    pred_val_q4=np.dot(X_shuf_val[:,(19-c):],v_train_fit)
    
    mse_train_q4[c]=np.mean((y_shuf_train-pred_train_q4)**2)
    mse_val_q4[c]=np.mean((y_shuf_val-pred_val_q4)**2)
    
index_c_tra=np.where(mse_train_q4==np.min(mse_train_q4))[0].item()
index_c_val=np.where(mse_val_q4==np.min(mse_val_q4))[0].item()
print('Training set:\nMSE={0:.3e} \t C={1:2d}'.format(np.min(mse_train_q4),index_c_tra+1)) 
print('Validation set:\nMSE={0:.3e} \t C={1:2d}'.format(np.min(mse_val_q4),index_c_val+1)) 

### Question4_b
- Results

In [ ]:
v_train_fit_q4b=np.linalg.lstsq(X_shuf_train[:,2:],y_shuf_train,rcond=None)[0]
pred_test_q4=np.dot(X_shuf_test[:,2:],v_train_fit_q4b)
mse_test_q4=np.mean((y_shuf_test-pred_test_q4)**2)
print('MSE in q4: {0:2e}\nMSE in q3: {1:2e}'.format(mse_test_q4,mse_test))

pred_q4c=np.dot(X_shuf_val[:,:20],np.linalg.lstsq(X_shuf_val[:,:20],y_shuf_val,rcond=None)[0])
res=y_shuf_val-pred_q4c
plt.hist(res,bins=100)
plt.title('Residuals on the validation data')
plt.xlabel('Residuals')
plt.ylabel('Count')
plt.show()